<a href="https://colab.research.google.com/github/37stu37/Ensemble_earthquake_Nepal/blob/main/remotness_analysis_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from IPython.display import display, HTML

# Set the style for our plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = [12, 8]


In [2]:
!wget https://github.com/37stu37/Ensemble_earthquake_Nepal/blob/main/Remoteness_DFID_Data.csv

--2025-02-28 16:23:18--  https://github.com/37stu37/Ensemble_earthquake_Nepal/blob/main/Remoteness_DFID_Data.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Remoteness_DFID_Data.csv’

Remoteness_DFID_Dat     [  <=>               ] 222.00K   694KB/s    in 0.3s    

2025-02-28 16:23:19 (694 KB/s) - ‘Remoteness_DFID_Data.csv’ saved [227324]



In [3]:
# Define time categories in correct order
time_categories = [
    "0 to 30 minutes",
    "30 minutes to 1 hour",
    "1 to 2 hours",
    "2 to 4 hours",
    "4 to 8 hours",
    "8 to 16 hours",
    "16 to 32 hours",
    "> 32 hours"
]

# Create a mapping for categories to ordinal positions
time_category_order = {cat: i for i, cat in enumerate(time_categories)}

# Define facility types we're interested in
facility_types = ["Health posts and sub-health posts", "District Headquarters"]

try:
    df = pd.read_csv('/content/Remoteness_DFID_Data.csv')
except FileNotFoundError:
    df = pd.read_csv('../../data/Remotenessdata/Remoteness_DFID_Data.csv')

# Display the first few rows
print("Dataset shape:", df.shape)
print(df.head())

# %%
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

# Display unique values for key columns
print("Unique facility types:")
print(df['fac_type'].unique())

print("\nUnique seasons:")
print(df['season'].unique())

print("\nUnique travel categories:")
print(df['trav_cat'].unique())

print("\nNumber of unique municipalities:", df['adm_name'].nunique())


ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 44


In [ ]:
# Filter for Normal season and our two facility types
filtered_df = df[
    (df['season'] == 'Normal season') &
    (df['fac_type'].isin(facility_types))
]

print("Filtered dataset shape:", filtered_df.shape)

# Check the count by facility type
facility_counts = filtered_df['fac_type'].value_counts()
print("\nCount by facility type:")
print(facility_counts)

Filtered dataset shape: (7594, 15)

Count by facility type:
fac_type
Health posts and sub-health posts    4141
District Headquarters                3453
Name: count, dtype: int64


In [ ]:
# ## Creating Municipality Rankings

# %%
# Create rankings for each time category and facility type
rankings = {}

for facility in facility_types:
    facility_short = "HealthPost" if facility == "Health posts and sub-health posts" else "DistrictHQ"
    rankings[facility_short] = {}

    for category in time_categories:
        # Get data for this facility type and time category
        category_data = filtered_df[
            (filtered_df['fac_type'] == facility) &
            (filtered_df['trav_cat'] == category)
        ]

        # Rank municipalities by tc_pc_pop (percentage of population)
        # Higher percentage = better rank (rank 1 is best)
        ranked = category_data.sort_values('tc_pc_pop', ascending=False)
        ranked['rank'] = range(1, len(ranked) + 1)

        # Store as a dictionary for easy lookup
        rankings[facility_short][category] = dict(zip(ranked['adm_name'], ranked['rank']))

In [ ]:
# ## Correlation Analysis Between Rankings

# %%
# Calculate correlation between rankings in different time categories
correlation_matrices = {}

for facility in ["HealthPost", "DistrictHQ"]:
    correlation_matrices[facility] = pd.DataFrame(index=time_categories, columns=time_categories)

    for cat1 in time_categories:
        for cat2 in time_categories:
            # Get municipalities that have rankings in both categories
            rank1 = rankings[facility][cat1]
            rank2 = rankings[facility][cat2]

            common_munis = set(rank1.keys()).intersection(set(rank2.keys()))

            if len(common_munis) > 10:  # Ensure enough data points
                # Extract rankings
                rank1_values = [rank1[muni] for muni in common_munis]
                rank2_values = [rank2[muni] for muni in common_munis]

                # Calculate correlation
                correlation = np.corrcoef(rank1_values, rank2_values)[0, 1]
                correlation_matrices[facility].loc[cat1, cat2] = correlation
            else:
                correlation_matrices[facility].loc[cat1, cat2] = np.nan

# Convert to numeric values
for facility in correlation_matrices:
    correlation_matrices[facility] = correlation_matrices[facility].astype(float)

# Print correlation results for the first few time categories
print("Correlation between time categories for Health Posts:")
print(correlation_matrices["HealthPost"].iloc[:4, :4].round(2))

print("\nCorrelation between time categories for District Headquarters:")
print(correlation_matrices["DistrictHQ"].iloc[:4, :4].round(2))

Correlation between time categories for Health Posts:
                      0 to 30 minutes  30 minutes to 1 hour  1 to 2 hours  \
0 to 30 minutes                  1.00                 -0.28         -0.78   
30 minutes to 1 hour            -0.28                  1.00          0.51   
1 to 2 hours                    -0.78                  0.51          1.00   
2 to 4 hours                    -0.86                 -0.04          0.66   

                      2 to 4 hours  
0 to 30 minutes              -0.86  
30 minutes to 1 hour         -0.04  
1 to 2 hours                  0.66  
2 to 4 hours                  1.00  

Correlation between time categories for District Headquarters:
                      0 to 30 minutes  30 minutes to 1 hour  1 to 2 hours  \
0 to 30 minutes                  1.00                  0.00         -0.66   
30 minutes to 1 hour             0.00                  1.00         -0.20   
1 to 2 hours                    -0.66                 -0.20          1.00   
2 t

In [ ]:
# ## Finding Extreme-Ranked Municipalities

# %%
# Let's get the top and bottom ranked municipalities in the shortest category
def get_extreme_ranked_municipalities(facility, category, n=5):
    facility_key = "HealthPost" if facility == "Health posts and sub-health posts" else "DistrictHQ"

    # Sort by rank
    sorted_munis = sorted(rankings[facility_key][category].items(), key=lambda x: x[1])

    # Get top and bottom n
    top_n = [muni for muni, rank in sorted_munis[:n]]
    bottom_n = [muni for muni, rank in sorted_munis[-n:]]

    return top_n, bottom_n

# Get top and bottom municipalities for the shortest time category
top_hp, bottom_hp = get_extreme_ranked_municipalities("Health posts and sub-health posts", "0 to 30 minutes", 5)
top_dhq, bottom_dhq = get_extreme_ranked_municipalities("District Headquarters", "0 to 30 minutes", 5)

print("Top 5 municipalities for Health Posts (0-30 min):", top_hp)
print("Bottom 5 municipalities for Health Posts (0-30 min):", bottom_hp)

print("\nTop 5 municipalities for District HQ (0-30 min):", top_dhq)
print("Bottom 5 municipalities for District HQ (0-30 min):", bottom_dhq)


Top 5 municipalities for Health Posts (0-30 min): ['Nepalgunj', 'Madhyapur Thimi', 'Bhaktapur', 'Lumbini Sanskritik Development Area', 'Kathmandu']
Bottom 5 municipalities for Health Posts (0-30 min): ['Kaike', 'Apihimal', 'Kanda', 'Himali', 'Shey Phoksundo']

Top 5 municipalities for District HQ (0-30 min): ['Kathmandu', 'Bhaktapur', 'Lalitpur', 'Madhyapur Thimi', 'Kirtipur']
Bottom 5 municipalities for District HQ (0-30 min): ['Nawarajpur', 'Bhanu', 'Barbardiya', 'Mahakali', 'Arun']


In [ ]:
# Include some from top/bottom and known interesting cases
interesting_munis = {
    "HealthPost": list(set(top_hp + bottom_hp + ["Madi", "Dolpo Buddha", "Mahalaxmi", "Kalika", "Mayadevi"])),
    "DistrictHQ": list(set(top_dhq + bottom_dhq + ["Bhaktapur", "Mahalaxmi", "Likhu", "Dolpo Buddha", "Mayadevi"]))
}

# %%
# Create function to get rankings for specific municipalities
def get_municipality_rankings(facility_type, municipalities):
    facility_key = "HealthPost" if facility_type == "Health posts and sub-health posts" else "DistrictHQ"

    # Create a dataframe with rankings for each municipality across time categories
    data = []

    for muni in municipalities:
        muni_data = {'Municipality': muni}

        for category in time_categories:
            if muni in rankings[facility_key][category]:
                muni_data[category] = rankings[facility_key][category][muni]
            else:
                muni_data[category] = np.nan

        data.append(muni_data)

    return pd.DataFrame(data)

# Get rankings for our interesting municipalities
health_post_rankings = get_municipality_rankings("Health posts and sub-health posts", interesting_munis["HealthPost"])
district_hq_rankings = get_municipality_rankings("District Headquarters", interesting_munis["DistrictHQ"])

# Display the rankings
print("Health Post Rankings for Key Municipalities:")
print(health_post_rankings)

print("\nDistrict HQ Rankings for Key Municipalities:")
print(district_hq_rankings)


Health Post Rankings for Key Municipalities:
                           Municipality  0 to 30 minutes  \
0                                 Kanda            758.0   
1                                  Madi            623.0   
2                                Himali            759.0   
3                              Apihimal            757.0   
4                             Mahalaxmi            607.0   
5                          Dolpo Buddha              NaN   
6                             Kathmandu              5.0   
7                             Bhaktapur              3.0   
8                             Nepalgunj              1.0   
9                                 Kaike            756.0   
10  Lumbini Sanskritik Development Area              4.0   
11                             Mayadevi            136.0   
12                               Kalika            709.0   
13                       Shey Phoksundo            760.0   
14                      Madhyapur Thimi              2.

In [ ]:
# ## Interactive Visualizations

# %% [markdown]
# ### 1. Interactive Line Chart for Health Post Rankings

# %%
# Interactive line chart for Health Post rankings
fig = go.Figure()

for idx, row in health_post_rankings.iterrows():
    muni = row['Municipality']
    # Get category values and replace NaN with None for plotting
    values = [row[cat] if not pd.isna(row[cat]) else None for cat in time_categories]

    # Create a list for x values with None where y is None
    x_values = list(range(len(time_categories)))

    # Add line to plot
    fig.add_trace(go.Scatter(
        x=x_values,
        y=values,
        mode='lines+markers',
        name=muni,
        hovertemplate='<b>%s</b><br>Time: %s<br>Rank: %%{y}<extra></extra>' % (muni, '%{text}'),
        text=time_categories
    ))

# Configure the layout
fig.update_layout(
    title='Health Post Rankings Across Time Categories',
    xaxis=dict(
        title='Time Category',
        tickmode='array',
        tickvals=list(range(len(time_categories))),
        ticktext=time_categories
    ),
    yaxis=dict(
        title='Rank (lower is better)',
        autorange='reversed'  # Invert y-axis so lower ranks (better) are at the top
    ),
    hovermode='closest',
    legend=dict(
        orientation='v',
        yanchor='top',
        y=0.99,
        xanchor='left',
        x=1.02
    ),
    width=900,
    height=600,
    margin=dict(l=20, r=200, t=40, b=40)
)

fig.show(renderer="colab")


In [ ]:
# ### 2. Interactive Line Chart for District HQ Rankings

# %%
# Interactive line chart for District HQ rankings
fig = go.Figure()

for idx, row in district_hq_rankings.iterrows():
    muni = row['Municipality']
    # Get category values and replace NaN with None for plotting
    values = [row[cat] if not pd.isna(row[cat]) else None for cat in time_categories]

    # Create a list for x values with None where y is None
    x_values = list(range(len(time_categories)))

    # Add line to plot
    fig.add_trace(go.Scatter(
        x=x_values,
        y=values,
        mode='lines+markers',
        name=muni,
        hovertemplate='<b>%s</b><br>Time: %s<br>Rank: %%{y}<extra></extra>' % (muni, '%{text}'),
        text=time_categories
    ))

# Configure the layout
fig.update_layout(
    title='District Headquarters Rankings Across Time Categories',
    xaxis=dict(
        title='Time Category',
        tickmode='array',
        tickvals=list(range(len(time_categories))),
        ticktext=time_categories
    ),
    yaxis=dict(
        title='Rank (lower is better)',
        autorange='reversed'  # Invert y-axis so lower ranks (better) are at the top
    ),
    hovermode='closest',
    legend=dict(
        orientation='v',
        yanchor='top',
        y=0.99,
        xanchor='left',
        x=1.02
    ),
    width=900,
    height=600,
    margin=dict(l=20, r=200, t=40, b=40)
)

fig.show()


In [ ]:
# ### 3. Interactive Heatmaps for Correlation Matrices

# %%
# Interactive heatmap for Health Post correlation matrix
fig = px.imshow(
    correlation_matrices["HealthPost"],
    x=time_categories,
    y=time_categories,
    color_continuous_scale=px.colors.diverging.RdBu_r,  # Red-Blue diverging colorscale
    range_color=[-1, 1],  # Set range from -1 to 1
    labels=dict(color="Correlation"),
    title="Correlation Between Time Categories for Health Posts"
)

# Add text annotations
for i in range(len(time_categories)):
    for j in range(len(time_categories)):
        value = correlation_matrices["HealthPost"].iloc[i, j]
        if not pd.isna(value):
            fig.add_annotation(
                x=j,
                y=i,
                text=f"{value:.2f}",
                showarrow=False,
                font=dict(color="black" if abs(value) < 0.6 else "white")
            )

fig.update_layout(width=800, height=700)
fig.show()


In [ ]:
# Interactive heatmap for District HQ correlation matrix
fig = px.imshow(
    correlation_matrices["DistrictHQ"],
    x=time_categories,
    y=time_categories,
    color_continuous_scale=px.colors.diverging.RdBu_r,  # Red-Blue diverging colorscale
    range_color=[-1, 1],  # Set range from -1 to 1
    labels=dict(color="Correlation"),
    title="Correlation Between Time Categories for District Headquarters"
)

# Add text annotations
for i in range(len(time_categories)):
    for j in range(len(time_categories)):
        value = correlation_matrices["DistrictHQ"].iloc[i, j]
        if not pd.isna(value):
            fig.add_annotation(
                x=j,
                y=i,
                text=f"{value:.2f}",
                showarrow=False,
                font=dict(color="black" if abs(value) < 0.6 else "white")
            )

fig.update_layout(width=800, height=700)
fig.show()


In [ ]:
# ### 5. Interactive Distribution of Population Percentage

# %%
# Interactive distribution plots for Health Posts
facility = "Health posts and sub-health posts"

# Get the data for this facility type
facility_data = filtered_df[filtered_df['fac_type'] == facility]

# Create subplot grid
fig = make_subplots(rows=2, cols=4, subplot_titles=time_categories)

for i, category in enumerate(time_categories):
    row = (i // 4) + 1
    col = (i % 4) + 1

    # Get data for this category
    cat_data = facility_data[facility_data['trav_cat'] == category]

    # Create a histogram
    fig.add_trace(
        go.Histogram(
            x=cat_data['tc_pc_pop'],
            nbinsx=30,
            marker_color='skyblue',
            name=category,
            hovertemplate='Population %: %{x}<br>Count: %{y}<extra></extra>'
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text='Distribution of Population Percentage by Time Category - Health Posts',
    height=800,
    width=1100,
    showlegend=False
)

fig.show()


In [ ]:
# Interactive distribution plots for District Headquarters
facility = "District Headquarters"

# Get the data for this facility type
facility_data = filtered_df[filtered_df['fac_type'] == facility]

# Create subplot grid
fig = make_subplots(rows=2, cols=4, subplot_titles=time_categories)

for i, category in enumerate(time_categories):
    row = (i // 4) + 1
    col = (i % 4) + 1

    # Get data for this category
    cat_data = facility_data[facility_data['trav_cat'] == category]

    # Create a histogram
    fig.add_trace(
        go.Histogram(
            x=cat_data['tc_pc_pop'],
            nbinsx=30,
            marker_color='lightgreen',
            name=category,
            hovertemplate='Population %: %{x}<br>Count: %{y}<extra></extra>'
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text='Distribution of Population Percentage by Time Category - District Headquarters',
    height=800,
    width=1100,
    showlegend=False
)

fig.show()


In [ ]:
# Create interactive dropdowns to compare municipalities
def create_municipality_comparison_plot():
    # Get all municipalities that have data for both facility types
    all_munis = set()
    for facility in ["HealthPost", "DistrictHQ"]:
        for category in time_categories:
            all_munis.update(rankings[facility][category].keys())

    all_munis = sorted(list(all_munis))

    # Create a dropdown widget
    from ipywidgets import interact, widgets

    @interact(
        municipality=widgets.Dropdown(options=all_munis, description='Municipality:'),
        facility=widgets.Dropdown(options=["Health Posts", "District HQ", "Both"], description='Facility Type:')
    )
    def plot_municipality_rankings(municipality, facility):
        fig = go.Figure()

        # Get data for health posts
        hp_data = []
        for cat in time_categories:
            if municipality in rankings["HealthPost"].get(cat, {}):
                hp_data.append({
                    'category': cat,
                    'rank': rankings["HealthPost"][cat][municipality],
                    'cat_index': time_category_order[cat]
                })
        hp_df = pd.DataFrame(hp_data).sort_values('cat_index')

        # Get data for district HQ
        dhq_data = []
        for cat in time_categories:
            if municipality in rankings["DistrictHQ"].get(cat, {}):
                dhq_data.append({
                    'category': cat,
                    'rank': rankings["DistrictHQ"][cat][municipality],
                    'cat_index': time_category_order[cat]
                })
        dhq_df = pd.DataFrame(dhq_data).sort_values('cat_index')

        # Plot based on selection
        if facility in ["Health Posts", "Both"]:
            fig.add_trace(go.Scatter(
                x=hp_df['category'],
                y=hp_df['rank'],
                mode='lines+markers',
                name='Health Posts',
                line=dict(color='blue', width=3),
                marker=dict(size=10)
            ))

        if facility in ["District HQ", "Both"]:
            fig.add_trace(go.Scatter(
                x=dhq_df['category'],
                y=dhq_df['rank'],
                mode='lines+markers',
                name='District HQ',
                line=dict(color='green', width=3),
                marker=dict(size=10)
            ))

        fig.update_layout(
            title=f'Municipality Rankings: {municipality}',
            xaxis_title='Time Category',
            yaxis_title='Rank (lower is better)',
            yaxis=dict(autorange="reversed"),
            height=500,
            width=900
        )

        fig.show()

    return plot_municipality_rankings

# Create the interactive widget (will display below this cell when run)
municipality_plot = create_municipality_comparison_plot()



interactive(children=(Dropdown(description='Municipality:', options=('Aamchowk', 'Aandhikhola', 'Aarughat', 'A…

In [ ]:
# Create interactive facility comparison for specific time categories
def create_facility_comparison():
    """Compare Health Posts and District HQ rankings for a specific time category"""
    from ipywidgets import interact, widgets

    @interact(
        category=widgets.Dropdown(options=time_categories, description='Time Category:')
    )
    def plot_facility_comparison(category):
        # Get municipalities that have both types of rankings in the selected category
        common_munis = set(rankings["HealthPost"][category].keys()).intersection(
            set(rankings["DistrictHQ"][category].keys())
        )

        # Create dataframe for scatter plot
        scatter_data = []
        for muni in common_munis:
            scatter_data.append({
                'Municipality': muni,
                'HealthPost': rankings["HealthPost"][category][muni],
                'DistrictHQ': rankings["DistrictHQ"][category][muni]
            })

        scatter_df = pd.DataFrame(scatter_data)

        # Calculate correlation
        corr = np.corrcoef(scatter_df['HealthPost'], scatter_df['DistrictHQ'])[0, 1]

        # Create scatter plot
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=scatter_df['HealthPost'],
            y=scatter_df['DistrictHQ'],
            mode='markers',
            marker=dict(size=10, opacity=0.7),
            text=scatter_df['Municipality'],
            hovertemplate='<b>%{text}</b><br>Health Post Rank: %{x}<br>District HQ Rank: %{y}<extra></extra>'
        ))

        # Add a reference line
        max_val = max(scatter_df['HealthPost'].max(), scatter_df['DistrictHQ'].max())
        fig.add_trace(go.Scatter(
            x=[0, max_val],
            y=[0, max_val],
            mode='lines',
            line=dict(color='gray', dash='dash'),
            showlegend=False
        ))

        fig.update_layout(
            title=f'Health Post vs District HQ Rankings for {category}<br>Correlation: {corr:.2f}',
            xaxis=dict(
                title='Health Post Rank',
                autorange='reversed'
            ),
            yaxis=dict(
                title='District HQ Rank',
                autorange='reversed'
            ),
            height=600,
            width=800
        )

        # Highlight points where there's a big difference in ranks
        diff_threshold = scatter_df['HealthPost'].max() * 0.25  # Set a threshold at 25% of max rank
        outliers = scatter_df[abs(scatter_df['HealthPost'] - scatter_df['DistrictHQ']) > diff_threshold]

        if not outliers.empty:
            fig.add_trace(go.Scatter(
                x=outliers['HealthPost'],
                y=outliers['DistrictHQ'],
                mode='markers+text',
                marker=dict(size=12, color='red', opacity=0.9),
                text=outliers['Municipality'],
                textposition="top center",
                name='Large Difference',
                hovertemplate='<b>%{text}</b><br>Health Post Rank: %{x}<br>District HQ Rank: %{y}<extra></extra>'
            ))

        fig.show()

        # Show municipalities with large differences in a table
        if not outliers.empty:
            outliers['Rank Difference'] = abs(outliers['HealthPost'] - outliers['DistrictHQ'])
            outliers_sorted = outliers.sort_values('Rank Difference', ascending=False)
            print(f"\nMunicipalities with large differences in {category}:")
            print(outliers_sorted[['Municipality', 'HealthPost', 'DistrictHQ', 'Rank Difference']])

    return plot_facility_comparison

# Create the interactive widget
facility_comparison = create_facility_comparison()


interactive(children=(Dropdown(description='Time Category:', options=('0 to 30 minutes', '30 minutes to 1 hour…